## 사고 다발지역 데이터와 사고 다발지역이 아닌 데이터를 합치고 위험도 라벨링

In [1]:
accident_path = '../data/accident_80'
no_accident_path = '../data/no_accident_80_500'

In [2]:
import pandas as pd

accident_lonlat = pd.read_csv(accident_path + '/lon_lat_list.csv', encoding = 'cp949', index_col = 0, header = 0, engine = 'python' )
accident_lonlat.head()

,lon,lat,is_raw,id
사고다발지FID,,,,
104688,127.015644,37.573022,True,0
104683,127.009633,37.569467,True,1
104684,127.008737,37.571310,True,2
104573,127.002189,37.576176,True,3
104773,126.997298,37.570828,True,4


In [3]:
accident_score = '../data/17_21_이륜차_사고다발지역_위험도계산.csv'

accident_score = pd.read_csv(accident_score, encoding = 'cp949', index_col = 0, header = 0, engine = 'python' )
accident_score.head()

,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,위험도
사고다발지FID,,,,,,,
104688,43,53,0,19,27,7,2.4259
104683,28,33,0,6,16,11,0.8459
104684,25,28,0,6,13,9,0.8189
104573,24,24,0,9,12,3,1.1427
104773,21,31,0,12,16,3,1.5203


In [4]:
accident = pd.merge(left = accident_lonlat, right = accident_score, how = 'left', on = '사고다발지FID')
accident = accident.reset_index()
accident.head()

,사고다발지FID,lon,lat,is_raw,id,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,위험도
0,104688,127.015644,37.573022,True,0,43,53,0,19,27,7,2.4259
1,104683,127.009633,37.569467,True,1,28,33,0,6,16,11,0.8459
2,104684,127.008737,37.571310,True,2,25,28,0,6,13,9,0.8189
3,104573,127.002189,37.576176,True,3,24,24,0,9,12,3,1.1427
4,104773,126.997298,37.570828,True,4,21,31,0,12,16,3,1.5203


In [5]:
accident2 = accident.loc[:, ['id', 'lon', 'lat', '위험도']]
accident2.head()

,id,lon,lat,위험도
0,0,127.015644,37.573022,2.4259
1,1,127.009633,37.569467,0.8459
2,2,127.008737,37.571310,0.8189
3,3,127.002189,37.576176,1.1427
4,4,126.997298,37.570828,1.5203


In [6]:
no_accident = pd.read_csv(no_accident_path + '/lon_lat_list.csv', encoding = 'cp949', index_col = 0, header = 0, engine = 'python' )
no_accident=no_accident.reset_index()
no_accident['위험도'] = 0.0

no_accident = no_accident.sample(n=len(accident2), # or equivalently: frac=2
                                 replace=False, # random sampling with replacement
                                 random_state=1034)
print('len', len(no_accident))
no_accident.head()

len 10362


,id,lat,lon,위험도
6429,27867,37.400391,126.629753,0.0
5614,22769,35.196150,129.077660,0.0
1484,28106,37.392751,126.426165,0.0
999,41122,36.329919,127.347815,0.0
15590,39420,36.361372,127.346895,0.0


In [7]:
all_accident = pd.concat([accident2, no_accident], axis=0)
print(len(all_accident))
all_accident['index'] = all_accident.index
all_accident.head()

20724


,id,lon,lat,위험도,index
0,0,127.015644,37.573022,2.4259,0
1,1,127.009633,37.569467,0.8459,1
2,2,127.008737,37.571310,0.8189,2
3,3,127.002189,37.576176,1.1427,3
4,4,126.997298,37.570828,1.5203,4


In [9]:
data_path = '../data/for_cnn2/'
all_accident.to_csv(f'{data_path}/lon_lat_list_no_apply_is_raw.csv', encoding = 'cp949')